In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

T_f = 350.0

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFNSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=T_f)
abTempF2S = InterfaceData(size=inodes, value=T_f)
FluxS = InterfaceData(size=inodes, value=0.0)
FluxF = InterfaceData(size=inodes, value=0.0)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 8000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FNSR%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FNSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    # mapping
    FluxF.curr[:] = FluxS.curr[s2f]
    
    solverF.put_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=FluxF.curr
    )
    
    while True:
        # back up previous interface value
        FluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid robin interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        FluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        FluxF.curr[:] = FluxS.curr[s2f]
        
        # update residual
        FluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(FluxF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(FluxF)
                under_relax.update_solution(FluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=FluxF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-T_bot)/(T_f-T_bot)
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=21.
step=1, pc_iterations=21.
step=2, pc_iterations=19.
step=3, pc_iterations=20.
step=4, pc_iterations=17.
step=5, pc_iterations=19.
step=6, pc_iterations=16.
step=7, pc_iterations=18.
step=8, pc_iterations=17.
step=9, pc_iterations=17.
step=10, pc_iterations=17.
step=11, pc_iterations=16.
step=12, pc_iterations=17.
step=13, pc_iterations=17.
step=14, pc_iterations=14.
step=15, pc_iterations=16.
step=16, pc_iterations=14.
step=17, pc_iterations=16.
step=18, pc_iterations=14.
step=19, pc_iterations=16.
step=20, pc_iterations=14.
step=21, pc_iterations=14.
step=22, pc_iterations=14.
step=23, pc_iterations=14.
step=24, pc_iterations=14.
step=25, pc_iterations=14.
step=26, pc_iterations=14.
step=27, pc_iterations=14.
step=28, pc_iterations=14.
step=29, pc_iterations=14.
step=30, pc_iterations=14.
step=31, pc_iterations=14.
step=32, pc_iterations=14.
step=33, pc_iterations=14.
step=34, pc_iterations=13.
step=35, pc_iterations=14.
step=36, pc_iterations=13.
step=37, pc

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-T_bot)/(T_f-T_bot)
print(theta)
sio.savemat(
    'numeric_res/kFNSR%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.11363127 0.07232079 0.05992134 0.06050176 0.05932796 0.05696584
 0.05516671 0.05347258 0.05127572 0.05003818 0.04850487 0.04730735
 0.04577475 0.04462416 0.04333052 0.04235003 0.04113377 0.04012009
 0.03921482 0.03816174 0.03732087 0.03639764 0.03555658 0.03480091
 0.03397028 0.03321953 0.03244141 0.03178249 0.03118493 0.03053541
 0.02989416 0.02932876 0.02875305 0.02816535 0.02764897 0.02716979
 0.02664185 0.02616542 0.02567018 0.02524288 0.02483959 0.02444918
 0.02404231 0.02365557 0.02328212 0.02293621 0.02257123 0.02226442
 0.02196114 0.02162425 0.0213226  0.02105422 0.0207835  0.02050297
 0.02027527 0.02001826 0.01980512 0.01957981 0.01935246 0.01916785
 0.01899    0.01878577 0.01861022 0.01845444 0.01831524 0.01814533
 0.01803694 0.01790064 0.01780501 0.01770611 0.01759379 0.01752315
 0.01746738 0.01738463 0.01733791 0.01730136 0.017286   0.0172553
 0.01727903 0.01727643 0.01732263 0.01736558 0.01741952 0.01748431
 0.01757657 0.01771555 0.01781052 0.01797388 0.01817456 0.01834